#📌 Extracción

In [1]:
import requests
import pandas as pd
import io

# 1. URL "Raw" del archivo JSON en GitHub
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

try:
    print(f"🔄 Descargando datos desde: {url}...")
    
    # 2. Realizar la petición GET
    response = requests.get(url)
    response.raise_for_status()  # Lanza un error si la descarga falla
    
    # 3. Cargar los datos JSON
    data = response.json()
    
    # 4. Convertir a DataFrame
    # Como es probable que sea una lista de diccionarios, esto suele ser directo:
    df = pd.DataFrame(data)

    print("✅ ¡Datos cargados exitosamente!")
    print(f"📊 Dimensiones del dataset: {df.shape[0]} filas x {df.shape[1]} columnas")
    
    # 5. Mostrar las primeras filas
    display(df.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al conectar con GitHub: {e}")
except ValueError as e:
    print(f"❌ Error al interpretar el JSON: {e}")

🔄 Descargando datos desde: https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json...
✅ ¡Datos cargados exitosamente!
📊 Dimensiones del dataset: 7267 filas x 6 columnas


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

#📊 Carga y análisis

#📄Informe final